In [ ]:
import tensorflow as tf
import numpy as np
import paho.mqtt.client as mqtt

import os
import matplotlib.pyplot as plt
import PIL.Image as Image

In [ ]:
# Load the model
model = tf.keras.models.load_model('inception_augmented')

In [ ]:
# Set model requirements for image size
IMG_HEIGHT = 299
IMG_WIDTH = 299

# Set folder for testing images
test_drive="test_route"

In [ ]:
# MQTT Client details
# Broker
host = "broker" 
# Port
port = 1883
# Message topic
topic = "signs"

In [ ]:
# Define function to specify when the program is connected to host
def on_message(client,userdata, msg):
  try:
    print("message received!")
    # Throw Exception if there is an error
  except:
    print("Unexpected error:", sys.exc_info()[0])


client = mqtt.Client()
client.connect(host,port,60)
client.on_message = on_message

In [ ]:
def run_inference(file):
    """ Opens the img file, runs it through the classifier and returns a class number (int)."""
    sign = Image.open(test_drive+"/"+file).resize((IMG_WIDTH,IMG_HEIGHT))
    sign.show() # Display the image being tested for audience.
    sign  = np.array(sign)/255.0
    result = model.predict(sign[np.newaxis, ...])
    predicted_class = np.argmax(result[0], axis=-1)
    return predicted_class

In [ ]:
# Loop for running MQTT. 
# Open folder of images for testing
for img in os.listdir(test_drive):
    # Run prediction on image
    prediction = run_inference(img)
    # Publish the prediction onto the MQTT topic. Note it will arrive as type byte
    client.publish(topic,prediction)